In [14]:
import asyncio
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests


def get_chrome_options():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu") # Disable GPU acceleration
    chrome_options.add_argument("--window-size=1920x1080") # Set a default window size
    chrome_options.add_argument("--no-sandbox") # Bypass sandbox security
    chrome_options.add_argument("--user-agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'") # Set a user-agent
    return chrome_options

def get_temperature():
    driver = webdriver.Chrome(options=get_chrome_options())
    url = 'https://weather.com/weather/tenday/l/San+Diego+CA?canonicalCityId=cb5c473781cc06501376639dce8f0823a99187dcb42c79471a4303c076d66452'
    driver.get(url)
    temperature_element = driver.find_element(By.CSS_SELECTOR, 'span.DailyContent--temp--1s3a7')
    temperature = temperature_element.text
    driver.quit()
    return temperature

# Exapmle of using the requests library to get the temperature via the OpenWeatherMap API
def get_temperature_via_api(lat, lon, api_key):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
    response = requests.get(url)
    weather_data = response.json()
    temperature_kelvin = weather_data['main']['temp'] # Accessing the temperature from the 'main' key
    temperature_celsius = temperature_kelvin - 273.15 # Convert from Kelvin to Celsius
    temperature_fahrenheit = temperature_celsius * 9/5 + 32 # Convert from Celsius to Fahrenheit
    return temperature_fahrenheit

def get_stock_price():
    driver = webdriver.Chrome(options=get_chrome_options())
    url = 'https://finance.yahoo.com/quote/AAPL/'
    driver.get(url)
    stock_price_element = driver.find_element(By.CSS_SELECTOR, 'fin-streamer[data-symbol="AAPL"]')
    stock_price = stock_price_element.get_attribute('value')
    driver.quit()
    return stock_price

def get_current_time():
    driver = webdriver.Chrome(options=get_chrome_options())
    url = 'https://www.timeanddate.com/worldclock/usa/los-angeles'
    driver.get(url)
    time_element = driver.find_element(By.ID, 'ct')
    current_time = time_element.text
    driver.quit()
    return current_time

async def main():
    loop = asyncio.get_event_loop()
    temperature = await loop.run_in_executor(None, get_temperature)
    stock_price = await loop.run_in_executor(None, get_stock_price)
    print("Temperature:", temperature)
    print("Apple Stock Price:", stock_price)

async def run_async(api_key):
    while True: # Run the loop indefinitely
        loop = asyncio.get_event_loop()
        # temperature = await loop.run_in_executor(None, get_temperature_via_api, 32.71, 117.16, api_key) #add in real api key
        temperature = await loop.run_in_executor(None, get_temperature)
        stock_price = await loop.run_in_executor(None, get_stock_price)
        current_time = await loop.run_in_executor(None, get_current_time)
        print(f"Time: {current_time} | Temperature: {round(float(temperature.split('°')[0]),2)}°F | Apple Stock Price: {stock_price}")
        # print(f"Time: {current_time} | Temperature: {round(float(temperature),2)}°F | Apple Stock Price: {stock_price}")

        await asyncio.sleep(60) # Wait for 60 seconds before the next iteration


In [15]:
await run_async(None)

Time: 5:28:17 am | Temperature: 77.0°F | Apple Stock Price: 177.79
Time: 5:30:12 am | Temperature: 77.0°F | Apple Stock Price: 177.79
